# Closed Loop ComCam Image Ingestion and Application of Correction

This notebook is used for the level 3 integration tests from test plan LVV-P81 (https://jira.lsstcorp.org/secure/Tests.jspa#/testPlan/LVV-P81) as part of test cylce LVV-C176 (https://jira.lsstcorp.org/secure/Tests.jspa#/testCycle/LVV-C176). The following tests are currently run as part of this notebook:

 - LVV-T2229 (https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2229)
 
Execution steps are separated by horizontal lines. Upon completion, save the notebook and its output as a pdf file to be attached to the test execution in JIRA. 

Last executed by B. Quint

***
Run the setup.ipnyb notebook to bring all components up and in their enabled position. Check Chronograph.

***

Bring ComCom online and tranistion it to EnabledState. Check Chronograph.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import rubin_jupyter_utils.lab.notebook as nb
nb.utils.get_node()

In [ ]:
import os
import sys
import asyncio
import logging

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel import MTCS, ComCam
from lsst.ts.observatory.control import RotType

In [ ]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)

In [ ]:
log = logging.getLogger("setup")
log.level = logging.DEBUG

In [ ]:
domain = salobj.Domain()

In [ ]:
mtcs = MTCS(domain=domain, log=log)
mtcs.set_rem_loglevel(40)

In [ ]:
await mtcs.start_task

In [ ]:
comcam = ComCam(domain=domain, log=log)

In [ ]:
comcam.set_rem_loglevel(40)

In [ ]:
await comcam.start_task

In [ ]:
await comcam.enable()

---

Find a target around az = 120º and el = 60º and rotator angle at PhysicalSky and 1.8º.

At this position, the rotator stays within a couple of degrees of its initial position. This is because the CCW is not running (MTmount in simulation mode).

target -> az = 120$^o$, el = 60$^o$  

In [ ]:
target = await mtcs.find_target(az=120, el=60, mag_limit=8)

print(f"Target: {target}")

***
Slew to target:

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=1.9)

***
Once the different components are ready (M1M3, M2, rotator and CCW, hexapods) and tracking, take an image using the take_image command in playback mode.
This second image should be the one that uses the correction calculated with the first slew.

In [ ]:
exp_focus = await comcam.take_object(15)
print(f"Target exposure: {exp_focus}")

----
Using the Camera Hexapod, piston ComCam +1mm

In [ ]:
await mtcs.rem.mthexapod_1.cmd_offset.set_start(z=1000.)

----
While tracking, take an image with ComCam and check that the header is containing the right telemetry

In [ ]:
exp_intra = await comcam.take_object(15)
print(f"Target 1 exposure: {exp_intra}")

---
Using the Camera Hexapod, piston ComCam to -1mm

In [ ]:
await mtcs.rem.mthexapod_1.cmd_offset.set_start(z=-2000.)

---
While tracking, take an image with ComCam and check that the header is containing the right telemetry.

In [ ]:
exp_extra = await comcam.take_object(15)
print(f"Target 1 exposure: {exp_extra}")

---
Put the hexapod back to 0mm.

In [ ]:
await mtcs.rem.mthexapod_1.cmd_offset.set_start(z=1000.)

---
If using MTMount Simulator and CCW Following Mode Disabled, stop tracking to prevent the Rotator to hit the limit switches.

In [ ]:
await mtcs.stop_tracking()

---
Use the MTAOS to calculate the required offsets to be sent to M1M3, M2 and the hexapods

In [ ]:
await mtcs.rem.mtaos.cmd_runWEP.set_start(visitId=exp_intra[0], 
                                          extraId=exp_extra[0])

await mtcs.rem.mtaos.cmd_runOFC.start(timeout=60.)

await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

In [ ]:
await mtcs.set_state(
    state=salobj.State.STANDBY,
    settings=dict(mtaos="impg"),
    components=["mtaos"]
    )

In [ ]:
await mtcs.set_state(
    state=salobj.State.ENABLED,
    settings=dict(mtaos="impg"),
    components=["mtaos"]
    )

---
Process wavefront data

In [ ]:
await mtcs.rem.mtaos.cmd_runWEP.set_start(visitId=exp_intra[0] - 2021111900000, 
                                          extraId=exp_extra[0] - 2021111900000)

---
Apply the resulting offsets to the M1M3, M2 and the hexapods.

In [ ]:
await mtcs.rem.mtaos.cmd_runOFC.start(timeout=60.)

---
Issue the corrections

In [ ]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

***
Query the butler to verify that the images are there and check the metadata. This step must be verified using a separate noteboook. 

***
## Wrap Up and Shut Down

This cell is not currently included as part of the test execution, but included here as needed to shutdown the systems

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtaos"])

In [ ]:
await mtcs.lower_m1m3()

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtm1m3"])

In [ ]:
await mtcs.set_state(salobj.State.OFFLINE, components=["mtm1m3"])

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtm2"])

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mthexapod_1"])

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mthexapod_2"])

In [ ]:
await mtcs.standby()

In [ ]:
await comcam.standby()